# ResNet50

In [1]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

model = ResNet50(weights='imagenet')

img_path = '深度有趣/七、图片分类/img/elephant.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
# 将结果解码为元组列表 (class, description, probability)
# (一个列表代表批次中的一个样本）
print('Predicted:', decode_predictions(preds, top=3)[0])

Using TensorFlow backend.
Instructions for updating:
Colocations handled automatically by placer.
Predicted: [('n02504013', 'Indian_elephant', 0.79393536), ('n02504458', 'African_elephant', 0.106723815), ('n01871265', 'tusker', 0.094560444)]


# VGG16提取特征

In [2]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np

model = VGG16(weights='imagenet', include_top=False)

img_path = '深度有趣/七、图片分类/img/elephant.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = model.predict(x)

array([[[[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
          0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         [0.0000000e+00, 0.0000000e+00, 5.2116370e+00, ...,
          0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         [0.0000000e+00, 0.0000000e+00, 2.1714432e+00, ...,
          0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         ...,
         [0.0000000e+00, 0.0000000e+00, 1.5423770e+01, ...,
          0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
          0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
          0.0000000e+00, 7.9064183e+00, 0.0000000e+00]],

        [[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
          0.0000000e+00, 0.0000000e+00, 2.3707120e+00],
         [0.0000000e+00, 0.0000000e+00, 5.2500126e+01, ...,
          8.7676115e+00, 0.0000000e+00, 0.0000000e+00],
         [0.0000000e+00, 0.0000000e+00, 3.6414024e+01, .

# VGG19任意中间层抽取特征

In [5]:
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.models import Model
import numpy as np

base_model = VGG19(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('block4_pool').output)

block4_pool_features = model.predict(x)
block4_pool_features

array([[[[  26.919004,    0.      ,   72.109085, ...,  624.2513  ,
            81.156975,    0.      ],
         [   0.      ,    0.      ,   17.167263, ...,    0.      ,
             0.      ,    0.      ],
         [   0.      ,    0.      ,    0.      , ...,  178.7214  ,
           514.6346  ,    0.      ],
         ...,
         [   0.      ,   73.18804 ,    0.      , ...,    0.      ,
             0.      ,    0.      ],
         [ 502.54037 ,    0.      ,    0.      , ...,    0.      ,
             0.      ,    0.      ],
         [ 680.7063  ,    0.      ,   41.742336, ...,    0.      ,
             0.      ,    0.      ]],

        [[ 256.614   ,    0.      ,    0.      , ...,  107.14982 ,
            57.068684,    0.      ],
         [ 232.72995 ,    0.      ,    0.      , ...,    0.      ,
            35.70326 ,    0.      ],
         [   0.      ,    0.      ,    0.      , ...,  949.22424 ,
           509.77606 ,    0.      ],
         ...,
         [   0.      ,    0.      

# 在新类上微调InceptionV3

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K


# 构建不带分类器的预训练模型
base_model = InceptionV3(weights='imagenet', include_top=False)

# 添加全局平均池化层
x = base_model.output
x = GlobalAveragePooling2D()(x)

# 添加一个全连接
x = Dense(1024, activation='relu')(x)

# 添加分类器，假设有200个类
predictions = Dense(200, activation='softmax')(x)

# 构建需要训练的完整模型
model = Model(inputs=base_model.input, outputs=predictions)

# 首先，只训练顶部的几层
# 锁住所有InceptionV3的卷积层
for layer in base_model.layers:
    layer.trainable = False

# 编译模型
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# 在新的数据集上训练
model.fit_generator(···)